In [38]:
import sys
sys.path.append('build')

import numpy as np
import numpy.random as rnd

from pyversor import ega, cga
import pickle

In [39]:
import zmq
import random
import sys
import time

In [40]:
def CayleyLi(B):
    B_ = cga.Motor(0.0, B[0], B[1], B[2], B[3], B[4], B[5], 0.0)
    BB = B_ * B_
    Rp = cga.Motor(1.0, B[0], B[1], B[2], B[3], B[4], B[5], 0.0)
    R0 = cga.Motor(1.0 - BB[0], 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0)
    R4 = cga.Motor(0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, BB[7])
    Rn = R0 + R4
    Rden = R0 * R0 
    return (Rp * Rp * Rn * Rden.inverse())

def oexp(B):
    n = np.sqrt(1 + B[0] * B[0] + B[1] * B[1] + B[2] * B[2])
    s = B[0] * B[5] - B[1] * B[4] + B[2] * B[3]
    m = cga.Motor(1.0, B[0], B[1], B[2], B[3], B[4], B[5], s) * (1.0 / n)
    return m

In [41]:
ts = np.linspace(0, np.pi * 2, 100) / 2
screw = cga.DualLine(*rnd.randn(6)).unit()

In [42]:
def send(data):
    port = "5556"
    context = zmq.Context()
    socket = context.socket(zmq.PAIR)
    socket.connect("tcp://localhost:%s" % port)
    socket.send(data)
    socket.close()

In [43]:
def draw_exp():
    motors = [cga.generate.exp(t / 2.0 * screw) for t in ts]
    points = [ega.Vector(1,0,0).null().spin(motor) for motor in motors]
    data = pickle.dumps(points)
    send(data)

In [44]:
def draw_cayley():
    motors = [cga.generate.cayley(np.tan(t) * screw) for t in ts]
    points = [ega.Vector(1,0,0).null().spin(motor) for motor in motors]
    data = pickle.dumps(points)
    send(data)

In [45]:
def draw_oexp():
    motors = [cga.generate.outer_exp(t * screw) for t in ts]
    points = [ega.Vector(1,0,0).null().spin(motor) for motor in motors]
    data = pickle.dumps(points)
    send(data)

In [50]:
# draw_exp()


In [51]:
draw_cayley()


In [52]:
draw_oexp()